In [ ]:
from view_demo.utils import env_vars as evar

experiment_prefix = evar.EXPERIMENT_PREFIX
project_id = evar.PROJECT_ID
dataset_id = evar.DATASET_ID
table_id = evar.TABLE_ID
staging_bucket = evar.STAGING_BUCKET
location = evar.REGION
training_image_uri = evar.BASE_TRAINING_IMAGE
experiment_prefix = evar.EXPERIMENT_PREFIX
context_window = evar.CONTEXT_WINDOW
tf_serving_image = evar.TF_SERVING_IMAGE

run_id = f'context-window-{context_window}'

In [ ]:
from view_demo.train.custom_tf_trainer import trainer
from view_demo.preprocess.window import Generator as WindowGenerator
from view_demo.models.baseline import linear
experiment_tracking_on = staging_bucket is not None
trainer = trainer(
   project_id,
   location,
   dataset_id,
   table_id
)
trainer.read_dataset()
train_df, val_df, test_df = trainer.create_split()
wide_window = WindowGenerator(
    input_width=context_window, label_width=context_window, shift=1,
    label_columns=['T__degC'],
    train_df=train_df,
    test_df=test_df,
    val_df=val_df)
history = trainer.compile_and_fit(linear, wide_window)
if experiment_tracking_on:
    from google.cloud import aiplatform
    aiplatform.init(
        project=project_id,
        staging_bucket=staging_bucket,
        experiment=experiment_id
    )
    aiplatform.start_run(run=run_id)
    aiplatform.log_metrics({"val_loss": history.history['val_loss'][-1]})
    aiplatform.log_metrics({"val_mae": history.history['val_mean_absolute_error'][-1]})
    aiplatform.log_metrics({"train_loss": history.history['loss'][-1]})
    aiplatform.log_metrics({"train_mae": history.history['mean_absolute_error'][-1]})